In [60]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

In [61]:
import uproot
import numpy as np
from tqdm import tqdm as tqdm
import matplotlib.pyplot as plt
import os
import optparse
import os.path as osp
import awkward as ak
import math
import torch_geometric
import torch
import sys
#import pandas as pd
from torch_geometric.data import Data
import glob

In [86]:
# filename = "/grid_mnt/data__data.polcms/cms/sghosh/PIDREGDATA_CERN/CloseByPhoton_v5/histo/histo_3490989_0.root"

# filein = uproot.open(filename)["ticlDumper"]
        
        
        
# clusters = filein["clusters"].arrays()
# clusters = clusters[[x for x in ak.fields(clusters) if x not in ["run_","luminosityBlock_","event_"]]]
# tracksters = filein["trackstersMerged"].arrays()
# associations = filein["associations"].arrays()
# simtrackstersCP = filein["simtrackstersCP"].arrays()
# associations

In [88]:
def analyze(filename,nfile,threshold, output_dir):



        filein = uproot.open(filename)["ticlDumper"]
        
        
        
        clusters = filein["clusters"].arrays()
        clusters = clusters[[x for x in ak.fields(clusters) if x not in ["run_","luminosityBlock_","event_"]]]
        tracksters = filein["trackstersMerged"].arrays()
        associations = filein["associations"].arrays()
        simtrackstersCP = filein["simtrackstersCP"].arrays()

        ## flatten to get better shapes
        simToReco_CP = ak.flatten(associations.Mergetracksters_simToReco_CP)
        simToReco_CP_score = ak.flatten(associations.Mergetracksters_simToReco_CP_score)

        ## choose the one with best score | best for closeby photons/electrons 
        minsimscore_idxs = ak.argmin(simToReco_CP_score,axis=-1,keepdims=True )


        ## get corrsp trkster idxs
        matchtrk_idxs = simToReco_CP[minsimscore_idxs]

        tracksters.raw_energy[matchtrk_idxs]

        ratio_trksim = tracksters.raw_em_energy[matchtrk_idxs]/simtrackstersCP.raw_em_energy
        ratio_trksimtot = tracksters.raw_energy[matchtrk_idxs]/simtrackstersCP.raw_energy
        #ratio_trksim = ratio_trksim.to_numpy()

        #sel_clusters = clusters[tracksters.vertices_indexes[matchtrk_idxs]]
        ncalop = len(simtrackstersCP)



        listoftensors = []
        ##maskcalo = ak.argmin(sel_simmatch_scr[sel_simmatch_idxs == 0],axis=1,keepdims=True)
        for calop in range(ncalop):
            ntrks = len(tracksters.raw_energy[matchtrk_idxs][calop])
            if ntrks < 1:
                #print(ntrk)
                continue
            
            if ak.is_none(matchtrk_idxs[calop]):
                #print("was isnone skipped")
                continue
            
            #print(ntrks)
            for ntrk in range(ntrks):
                
                
                
                gun_feats = np.stack((simtrackstersCP.boundaryEta[calop],
                                      simtrackstersCP.boundaryPhi[calop],
                                      simtrackstersCP.raw_em_energy[calop],
                                      simtrackstersCP.raw_energy[calop],
                                      ratio_trksim[calop],
                                      ratio_trksimtot[calop],
                                      simToReco_CP_score[minsimscore_idxs][calop])).to_numpy().data.T

                #print(gun_feats.shape)
                #print(gun_feats[:,3])

                clus2d_feats = np.stack((ak.to_numpy(clusters.position_x[calop][ak.flatten(tracksters.vertices_indexes[matchtrk_idxs][calop][ntrk], axis=-1)]),
                                 ak.to_numpy(clusters.position_y[calop][ak.flatten(tracksters.vertices_indexes[matchtrk_idxs][calop][ntrk], axis=-1)]),
                                 ak.to_numpy(clusters.position_z[calop][ak.flatten(tracksters.vertices_indexes[matchtrk_idxs][calop][ntrk], axis=-1)]),
                                 ak.to_numpy(clusters.energy[calop][ak.flatten(tracksters.vertices_indexes[matchtrk_idxs][calop][ntrk], axis=-1)]),
                                 ak.to_numpy(clusters.cluster_time[calop][ak.flatten(tracksters.vertices_indexes[matchtrk_idxs][calop][ntrk], axis=-1)]),
                                 ak.to_numpy(clusters.cluster_layer_id[calop][ak.flatten(tracksters.vertices_indexes[matchtrk_idxs][calop][ntrk], axis=-1)]))).T


                #print(clus2d_feats.shape)
                #print(np.sum(clus2d_feats[:,3]))


                clus3d_feats = np.stack((tracksters.barycenter_eta[matchtrk_idxs][calop][ntrk],
                                         tracksters.barycenter_phi[matchtrk_idxs][calop][ntrk],
                                         tracksters.raw_em_energy[matchtrk_idxs][calop][ntrk],
                                         tracksters.raw_energy[matchtrk_idxs][calop][ntrk],
                                         min(ak.to_numpy(clusters.cluster_layer_id[calop][ak.flatten(tracksters.vertices_indexes[matchtrk_idxs][calop][ntrk], axis=-1)])),
                                         max(ak.to_numpy(clusters.cluster_layer_id[calop][ak.flatten(tracksters.vertices_indexes[matchtrk_idxs][calop][ntrk], axis=-1)])),
                                         tracksters.barycenter_x[matchtrk_idxs][calop][ntrk],
                                         tracksters.barycenter_y[matchtrk_idxs][calop][ntrk],
                                         tracksters.barycenter_z[matchtrk_idxs][calop][ntrk],
                                        )).T


                #print(clus3d_feats.shape)
                #print(clus3d_feats[3])


                datum = Data(clus2d_feat = torch.tensor(clus2d_feats, dtype=torch.float32),
                            clus3d_feat = torch.tensor(clus3d_feats, dtype=torch.float32),
                            gun_feat = torch.tensor(gun_feats, dtype=torch.float32))

                listoftensors.append(datum)
                    
                    

        processed_dir = output_dir #'/grid_mnt/data__data.polcms/cms/sghosh/NEWPID_DATA/PHO/' ## PHO
        import os
        import os.path as osp
        if not os.path.exists(processed_dir):
            os.makedirs(processed_dir)


        #torch.save(listoftensors, osp.join(processed_dir, 'data_{}_{}_{}_{}.pt'.format(nfile,evt,ngun,trkidx)))
        torch.save(listoftensors, osp.join(processed_dir, 'data_{}.pt'.format(nfile)))
        #print("skipped events:",nsskip)
        #print("average #tracksters:",np.mean(ntrkster))
        return 0

In [89]:
#raw_dir='/grid_mnt/data__data.polcms/cms/tarabini/GENPHOTESTPU2_noSmearing/step3_ticlv4_clue3d_CMSSW_12_4_0_pre2/'
#raw_dir="/grid_mnt/data__data.polcms/cms/cappati/240419_pions_step3/" ### PI
#raw_dir="/grid_mnt/data__data.polcms/cms/cappati/240517_photonsD98_step3/"
raw_dir="/grid_mnt/data__data.polcms/cms/sghosh/PIDREGDATA_CERN/CloseByPhoton_v5/histo/"
output_dir = '/grid_mnt/data__data.polcms/cms/sghosh/PIDREGDATA_CERN_PROCESSED/CloseByPhoton_07082024'
fnamelist = os.listdir(raw_dir)#+'STEP3_*.root'
print(fnamelist)

for idx, file in tqdm(enumerate(fnamelist)):
    try:
        #print(raw_dir+file,idx)
        analyze(raw_dir+file,idx, 0.6, output_dir) ### PHO
        #analyze(raw_dir+file,idx, 0.99) ### PI
        #break
    except:
        print("file is invalid, skipping....")

['histo_3490989_0.root', 'histo_3490989_1.root', 'histo_3490989_10.root', 'histo_3490989_100.root', 'histo_3490989_101.root', 'histo_3490989_102.root', 'histo_3490989_103.root', 'histo_3490989_104.root', 'histo_3490989_105.root', 'histo_3490989_106.root', 'histo_3490989_107.root', 'histo_3490989_108.root', 'histo_3490989_11.root', 'histo_3490989_111.root', 'histo_3490989_117.root', 'histo_3490989_12.root', 'histo_3490989_13.root', 'histo_3490989_14.root', 'histo_3490989_15.root', 'histo_3490989_16.root', 'histo_3490989_17.root', 'histo_3490989_18.root', 'histo_3490989_19.root', 'histo_3490989_2.root', 'histo_3490989_24.root', 'histo_3490989_3.root', 'histo_3490989_34.root', 'histo_3490989_35.root', 'histo_3490989_36.root', 'histo_3490989_39.root', 'histo_3490989_4.root', 'histo_3490989_42.root', 'histo_3490989_45.root', 'histo_3490989_47.root', 'histo_3490989_49.root', 'histo_3490989_5.root', 'histo_3490989_56.root', 'histo_3490989_6.root', 'histo_3490989_60.root', 'histo_3490989_62.ro

68it [18:01, 15.90s/it]


In [90]:
#raw_dir='/grid_mnt/data__data.polcms/cms/tarabini/GENPHOTESTPU2_noSmearing/step3_ticlv4_clue3d_CMSSW_12_4_0_pre2/'
#raw_dir="/grid_mnt/data__data.polcms/cms/cappati/240419_pions_step3/" ### PI
#raw_dir="/grid_mnt/data__data.polcms/cms/cappati/240517_photonsD98_step3/"
raw_dir="/grid_mnt/data__data.polcms/cms/sghosh/PIDREGDATA_CERN/CloseByElectron_v5/histo/"
output_dir = '/grid_mnt/data__data.polcms/cms/sghosh/PIDREGDATA_CERN_PROCESSED/CloseByElectron_07082024'
fnamelist = os.listdir(raw_dir)#+'STEP3_*.root'
print(fnamelist)

for idx, file in tqdm(enumerate(fnamelist)):
    try:
        #print(raw_dir+file,idx)
        analyze(raw_dir+file,idx, 0.6, output_dir) ### PHO
        #analyze(raw_dir+file,idx, 0.99) ### PI
        #break
    except:
        print("file is invalid, skipping....")

['histo_3490988_0.root', 'histo_3490988_1.root', 'histo_3490988_10.root', 'histo_3490988_100.root', 'histo_3490988_101.root', 'histo_3490988_102.root', 'histo_3490988_103.root', 'histo_3490988_104.root', 'histo_3490988_105.root', 'histo_3490988_107.root', 'histo_3490988_108.root', 'histo_3490988_109.root', 'histo_3490988_11.root', 'histo_3490988_110.root', 'histo_3490988_111.root', 'histo_3490988_112.root', 'histo_3490988_113.root', 'histo_3490988_114.root', 'histo_3490988_115.root', 'histo_3490988_116.root', 'histo_3490988_117.root', 'histo_3490988_118.root', 'histo_3490988_119.root', 'histo_3490988_12.root', 'histo_3490988_13.root', 'histo_3490988_14.root', 'histo_3490988_15.root', 'histo_3490988_16.root', 'histo_3490988_17.root', 'histo_3490988_18.root', 'histo_3490988_19.root', 'histo_3490988_2.root', 'histo_3490988_21.root', 'histo_3490988_22.root', 'histo_3490988_23.root', 'histo_3490988_24.root', 'histo_3490988_25.root', 'histo_3490988_3.root', 'histo_3490988_30.root', 'histo_34

67it [17:38, 15.80s/it]


In [91]:
#raw_dir='/grid_mnt/data__data.polcms/cms/tarabini/GENPHOTESTPU2_noSmearing/step3_ticlv4_clue3d_CMSSW_12_4_0_pre2/'
#raw_dir="/grid_mnt/data__data.polcms/cms/cappati/240419_pions_step3/" ### PI
#raw_dir="/grid_mnt/data__data.polcms/cms/cappati/240517_photonsD98_step3/"
raw_dir="/grid_mnt/data__data.polcms/cms/sghosh/PIDREGDATA_CERN/CloseByKaon_v5/histo/"
output_dir = '/grid_mnt/data__data.polcms/cms/sghosh/PIDREGDATA_CERN_PROCESSED/CloseByKaon_07082024'
fnamelist = os.listdir(raw_dir)#+'STEP3_*.root'
print(fnamelist)

for idx, file in tqdm(enumerate(fnamelist)):
    try:
        #print(raw_dir+file,idx)
        analyze(raw_dir+file,idx, 0.6, output_dir) ### PHO
        #analyze(raw_dir+file,idx, 0.99) ### PI
        #break
    except:
        print("file is invalid, skipping....")

['histo_3490987_0.root', 'histo_3490987_1.root', 'histo_3490987_10.root', 'histo_3490987_100.root', 'histo_3490987_101.root', 'histo_3490987_102.root', 'histo_3490987_103.root', 'histo_3490987_105.root', 'histo_3490987_106.root', 'histo_3490987_107.root', 'histo_3490987_108.root', 'histo_3490987_109.root', 'histo_3490987_110.root', 'histo_3490987_112.root', 'histo_3490987_113.root', 'histo_3490987_114.root', 'histo_3490987_115.root', 'histo_3490987_116.root', 'histo_3490987_117.root', 'histo_3490987_118.root', 'histo_3490987_119.root', 'histo_3490987_12.root', 'histo_3490987_13.root', 'histo_3490987_14.root', 'histo_3490987_15.root', 'histo_3490987_16.root', 'histo_3490987_17.root', 'histo_3490987_18.root', 'histo_3490987_19.root', 'histo_3490987_2.root', 'histo_3490987_20.root', 'histo_3490987_21.root', 'histo_3490987_22.root', 'histo_3490987_24.root', 'histo_3490987_25.root', 'histo_3490987_26.root', 'histo_3490987_27.root', 'histo_3490987_28.root', 'histo_3490987_29.root', 'histo_34

24it [06:09, 11.51s/it]

file is invalid, skipping....


35it [08:50, 11.41s/it]

file is invalid, skipping....


...

48it [11:47, 11.19s/it]

file is invalid, skipping....


76it [19:02, 11.62s/it]

file is invalid, skipping....


111it [28:22, 15.34s/it]


In [ ]:
#raw_dir='/grid_mnt/data__data.polcms/cms/tarabini/GENPHOTESTPU2_noSmearing/step3_ticlv4_clue3d_CMSSW_12_4_0_pre2/'
#raw_dir="/grid_mnt/data__data.polcms/cms/cappati/240419_pions_step3/" ### PI
#raw_dir="/grid_mnt/data__data.polcms/cms/cappati/240517_photonsD98_step3/"
raw_dir="/grid_mnt/data__data.polcms/cms/sghosh/PIDREGDATA_CERN/CloseByPion_v5/histo/"
output_dir = '/grid_mnt/data__data.polcms/cms/sghosh/PIDREGDATA_CERN_PROCESSED/CloseByPion_07082024'
fnamelist = os.listdir(raw_dir)#+'STEP3_*.root'
print(fnamelist)

for idx, file in tqdm(enumerate(fnamelist)):
    try:
        #print(raw_dir+file,idx)
        analyze(raw_dir+file,idx, 0.6, output_dir) ### PHO
        #analyze(raw_dir+file,idx, 0.99) ### PI
        #break
    except:
        print("file is invalid, skipping....")

['histo_3490990_0.root', 'histo_3490990_1.root', 'histo_3490990_10.root', 'histo_3490990_100.root', 'histo_3490990_101.root', 'histo_3490990_102.root', 'histo_3490990_103.root', 'histo_3490990_105.root', 'histo_3490990_107.root', 'histo_3490990_108.root', 'histo_3490990_109.root', 'histo_3490990_11.root', 'histo_3490990_110.root', 'histo_3490990_111.root', 'histo_3490990_112.root', 'histo_3490990_113.root', 'histo_3490990_114.root', 'histo_3490990_115.root', 'histo_3490990_116.root', 'histo_3490990_117.root', 'histo_3490990_118.root', 'histo_3490990_119.root', 'histo_3490990_12.root', 'histo_3490990_13.root', 'histo_3490990_15.root', 'histo_3490990_16.root', 'histo_3490990_17.root', 'histo_3490990_18.root', 'histo_3490990_19.root', 'histo_3490990_20.root', 'histo_3490990_21.root', 'histo_3490990_22.root', 'histo_3490990_23.root', 'histo_3490990_24.root', 'histo_3490990_25.root', 'histo_3490990_26.root', 'histo_3490990_27.root', 'histo_3490990_28.root', 'histo_3490990_29.root', 'histo_3

99it [26:14, 15.93s/it]